# Library and Data Preparation 

In [ ]:
import os
import fnmatch
import TWB.xliff as xliff
import spacy
import pandas as pd
import queue
import numpy as np

In [ ]:
# put your data path here
data_dir = '/home/mixalis/Downloads/Translators Without Borders/'
metadata = pd.read_excel(data_dir + 'Hackathon-for-Good-2019_TWB-Challenge_Metadata.xlsx')

# choose only text documents
# TODO The metadata contain some files multiple times and the table is almost 3 times the number of documents
# Using various combinations of subsets of the columns to get all the 12156 documents did not succeed
accepted_documents = metadata.loc[metadata['Format'] == 'doc'].drop_duplicates()

# update data path with the sdlxliff directory
data_dir += 'hackathon-for-good-2019_TWB-challenge_files/'

In [ ]:
# store all sdlxliff filenames into a list
document_names = []

# trying to get all doc documents based on extensions
for document in os.listdir(data_dir):    
    if fnmatch.fnmatch(document,'*.sdlxliff'):
        if fnmatch.fnmatch(document,'*.doc*'):
            document_names.append(data_dir + document)
        if fnmatch.fnmatch(document,'*.DOC*'):
            document_names.append(data_dir + document)
        if fnmatch.fnmatch(document,'*.txt*'):
            document_names.append(data_dir + document)    
        if fnmatch.fnmatch(document,'*.pdf*'):
            document_names.append(data_dir + document)
        if fnmatch.fnmatch(document,'*.PDF*'):
            document_names.append(data_dir + document)
        if fnmatch.fnmatch(document,'*.odt*'):
            document_names.append(data_dir + document)
        if fnmatch.fnmatch(document,'*.rtf*'):
            document_names.append(data_dir + document)
        if fnmatch.fnmatch(document,'*.dotx*'):
            document_names.append(data_dir + document)    

In [ ]:
# get the translated contents of all documents
document_contents = []

for document in list(document_names):
    # Avoid a couple of (probably) malformed xml documents(No root found by the parser)
    try:
        document_contents.append(xliff.XLIFF(document).target)
    except AttributeError:
        print(document)
        continue

In [ ]:
# FIFO queue
impact_queue = queue.Queue(len(document_names))

# Priority queue with tupes of (priority_number,data) as items, lowest valued entries are retrieved first
impact_queue2 = queue.PriorityQueue(len(document_names))

# Priority queue seems most proper for our use case
# Any change to the queue that does not concern the first element basically requires to create another queue